<a href="https://colab.research.google.com/github/ashimasinghal/Indian-NER/blob/main/urdu_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==2.6.0

     |████████████████████████████████| 540 kB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 36.7 MB/s 
     |████████████████████████████████| 5.6 MB 18.2 MB/s 
     |████████████████████████████████| 131 kB 35.1 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 8.2 MB 17.9 MB/s 
     |████████████████████████████████| 138 kB 42.3 MB/s 
     |████████████████████████████████| 127 kB 48.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.5
    Uninstalling transformers-4.12.5:
      Successfully uninstalled transformers-4.12.5
ERROR: pip's dependency resolv

In [ ]:
!pip install seqeval
!pip install transformers

In [ ]:

import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [ ]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [ ]:
!pip list | grep -E 'transformers|torch|Keras'

Keras-Preprocessing           1.1.2
torch                         1.10.0+cu111
torchaudio                    0.10.0+cu111
torchsummary                  1.5.1
torchtext                     0.11.0
torchvision                   0.11.1+cu111
transformers                  2.6.0


In [ ]:
# load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BERT/urdu/train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.columns = ['word', 'tag']

In [ ]:
df['word'] = df['word'].replace(np.nan, '.')
df.head(20)

,word,tag
0,لاہور,Location
1,(,Other
2,این,Other
3,این,Other
4,آئی,Other
5,),Other
6,اداکار,Other
7,وسیم,Person
8,عباس,Person
9,بھی,Other


In [ ]:
sentence_id = []
words = df['word']
sentence = 1
for word in words:
  sentence_id.append(sentence)
  if(word=='.'):
    sentence += 1

In [ ]:
df.insert(1,'Sentence ID',sentence_id,allow_duplicates=True)
df.head(50)

,word,Sentence ID,tag
0,لاہور,1,Location
1,(,1,Other
2,این,1,Other
3,این,1,Other
4,آئی,1,Other
5,),1,Other
6,اداکار,1,Other
7,وسیم,1,Person
8,عباس,1,Person
9,بھی,1,Other


In [ ]:
hindi = pd.DataFrame()
hindi = df[['Sentence ID','word','tag']]
hindi.rename(columns={'Sentence ID': 'Sent_ID','word': 'Word'}, inplace=True)
hindi

,Sent_ID,Word,tag
0,1,لاہور,Location
1,1,(,Other
2,1,این,Other
3,1,این,Other
4,1,آئی,Other
...,...,...,...
673020,24282,کو,Other
673021,24282,حتمی,Other
673022,24282,شکل,Other
673023,24282,دے,Other


In [ ]:
# hindi.POS.unique()

In [ ]:
hindi.tag.unique()

array(['Location', 'Other', 'Person', nan, 'Organization'], dtype=object)

In [ ]:
hindi['Sent_ID'].nunique(), hindi.Word.nunique(), hindi.tag.nunique()

(24282, 30865, 4)

In [ ]:

hindi.tag.value_counts()

Other           573557
Location         33130
Person           26615
Organization     15448
Name: tag, dtype: int64

In [ ]:
# parsing the data


In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# hindi_train, hindi_test = train_test_split( hindi, test_size=0.33, random_state=42)

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(hindi)

In [ ]:
sentences = [[s[0] for s in sent] for sent in getter.sentences]
sentences[0]

['لاہور',
 '(',
 'این',
 'این',
 'آئی',
 ')',
 'اداکار',
 'وسیم',
 'عباس',
 'بھی',
 'عارضی',
 'طور',
 'پر',
 'کراچی',
 'شفٹ',
 'ہو',
 'گئے',
 '.']

In [ ]:
# poses = [[s[1] for s in sent] for sent in getter.sentences]
# print(poses[0])

In [ ]:
labels = [[s[1] for s in sentence] for sentence in getter.sentences]
print(labels[0])

['Location', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Person', 'Person', 'Other', 'Other', 'Other', 'Other', 'Location', 'Other', 'Other', 'Other', nan]


In [ ]:
tag_values = list(set(df["tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.10.0+cu111'

In [ ]:
MAX_LEN = 75
bs = 32

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False) 

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'2.6.0'

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
model.cuda();

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
!pip install seqeval

In [ ]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        eval_accuracy += flat_accuracy(logits, label_ids)
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Average train loss: 0.18550134271296018


Epoch:   0%|          | 0/3 [10:31<?, ?it/s]

Validation loss: 0.13583711593558914
Validation Accuracy: 0.5477372580157288


TypeError: ignored

In [ ]:
# old codee

In [ ]:
tag2idx={'Location':1, 'Organization':2, 'Other':3, 'Person':4, 'X':5, '[CLS]':6, '[SEP]':7, 'nan':8
 }
tag2idx

{'Location': 1,
 'Organization': 2,
 'Other': 3,
 'Person': 4,
 'X': 5,
 '[CLS]': 6,
 '[SEP]': 7,
 'nan': 8}

In [ ]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [ ]:
# training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

1

In [ ]:
# vocabulary = "/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased/bert-base-multilingual-cased-vocab.txt"

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
max_len = 45 
tokenizer = AutoTokenizer.from_pretrained("skillzzzzzy/urberto")

Downloading:   0%|          | 0.00/671 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/795k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/596k [00:00<?, ?B/s]

In [ ]:
# max_len  = 45
# tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased', truncation=True, do_lower_case=True)
# tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
# model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")

In [ ]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

NameError: ignored

In [ ]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

IndexError: ignored

In [ ]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["o"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])


KeyError: ignored

In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

IndexError: ignored

In [ ]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

In [ ]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(6631, 2843, 6631, 2843)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [ ]:
batch_num = 32

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [ ]:
# model_file_address = '/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased'
model = AutoModelForMaskedLM.from_pretrained("skillzzzzzy/urberto", num_labels=len(tag2idx))
# model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased",num_labels=len(tag2idx))

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

In [ ]:
m_labels = len(tag2idx)

In [ ]:
model;

In [ ]:
model.cuda();

In [ ]:
epochs = 20
max_grad_norm = 1.0

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [ ]:
# fine tuning
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
model.train();

In [ ]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

***** Running training *****
  Num examples = 6631
  Batch size = 32
  Num steps = 4160


Epoch:   5%|▌         | 1/20 [01:24<26:46, 84.53s/it]

Train loss: 0.569116329225365


Epoch:  10%|█         | 2/20 [02:48<25:19, 84.40s/it]

Train loss: 0.16525093350865414


...

Train loss: 0.031232782464096512


Epoch:  95%|█████████▌| 19/20 [26:40<01:24, 84.25s/it]

Train loss: 0.02815081368555913


Epoch: 100%|██████████| 20/20 [28:05<00:00, 84.25s/it]

Train loss: 0.025632590289421126


In [ ]:
bert_out_address = 'ashiNLP/bert_hindberto_model/hindi'

In [ ]:
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model

In [ ]:
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [ ]:
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('ashiNLP/bert_hindberto_model/hindi/vocab.json',
 'ashiNLP/bert_hindberto_model/hindi/merges.txt')

In [ ]:
# load model
model = AutoModelForMaskedLM.from_pretrained(bert_out_address,num_labels=len(tag2idx))

In [ ]:
model.cuda();

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
# EVALUATION

In [ ]:
model.eval();

In [ ]:

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
# output_eval_file = os.path.join('/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased', "eval_results.txt")
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =2843
  Batch size = 32


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: o seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


f1 socre: 0.469522
Accuracy score: 0.855230


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** Eval results *****

               precision    recall  f1-score   support

     ARTIFACT     0.3333    0.2281    0.2708        57
        COUNT     0.7709    0.8412    0.8045       340
         DATE     0.0000    0.0000    0.0000         4
          DAY     0.0000    0.0000    0.0000         3
      DISEASE     0.2857    0.1739    0.2162        23
     DISTANCE     0.2000    0.2857    0.2353        14
ENTERTAINMENT     0.5455    0.4660    0.5026       103
   FACILITIES     0.0000    0.0000    0.0000        12
    LIVTHINGS     0.3571    0.2632    0.3030        19
     LOCATION     0.3909    0.4624    0.4236       372
   LOCOMOTIVE     1.0000    0.2857    0.4444         7
    MATERIALS     0.3000    0.2222    0.2553        27
        MONTH     0.4545    0.3333    0.3846        15
 ORGANIZATION     0.1875    0.1034    0.1333        29
       PERIOD     0.3235    0.4074    0.3607        27
       PERSON     0.5000    0.3833    0.4339       347
       PLANTS     0.0000    0.0000    

In [ ]:
save_model_address = 'ashiNLP/bert_hindberto_model/hindi'

In [ ]:
save_model = AutoModelForMaskedLM.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_model_address,do_lower_case=False)

In [ ]:
# predict
test_query = "मैं 20 साल से दिल्ली में रहता हूं"

In [ ]:
tokenized_texts = []

temp_token = []
temp_token.append('[CLS]')
token_list = tokenizer.tokenize(test_query)

token_list

['à¤',
 '®',
 'à¥',
 'Ī',
 'à¤',
 'Ĥ',
 'Ġ20',
 'Ġà¤',
 '¸',
 'à¤¾',
 'à¤',
 '²',
 'Ġà¤',
 '¸',
 'à¥ĩ',
 'Ġà¤',
 '¦',
 'à¤',
 '¿',
 'à¤',
 '²',
 'à¥į',
 'à¤',
 '²',
 'à¥Ģ',
 'Ġà¤',
 '®',
 'à¥ĩ',
 'à¤',
 'Ĥ',
 'Ġà¤',
 '°',
 'à¤',
 '¹',
 'à¤',
 '¤',
 'à¤¾',
 'Ġà¤',
 '¹',
 'à¥',
 'Ĥ',
 'à¤',
 'Ĥ']

In [ ]:
for m,token in enumerate(token_list):
    temp_token.append(token)

In [ ]:
if len(temp_token) > max_len-1:
    temp_token= temp_token[:max_len-1]

In [ ]:
temp_token.append('[SEP]')
temp_token

['[CLS]',
 'à¤',
 '®',
 'à¥',
 'Ī',
 'à¤',
 'Ĥ',
 'Ġ20',
 'Ġà¤',
 '¸',
 'à¤¾',
 'à¤',
 '²',
 'Ġà¤',
 '¸',
 'à¥ĩ',
 'Ġà¤',
 '¦',
 'à¤',
 '¿',
 'à¤',
 '²',
 'à¥į',
 'à¤',
 '²',
 'à¥Ģ',
 'Ġà¤',
 '®',
 'à¥ĩ',
 'à¤',
 'Ĥ',
 'Ġà¤',
 '°',
 'à¤',
 '¹',
 'à¤',
 '¤',
 'à¤¾',
 'Ġà¤',
 '¹',
 'à¥',
 'Ĥ',
 'à¤',
 'Ĥ',
 '[SEP]']

In [ ]:
tokenized_texts.append(temp_token)

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])


[    3  6843   111  9916   235  6843   229   734 14614   121 21507  6843
   115 14614   121 29892 14614   104  6843   128  6843   115 28023  6843
   115 30287 14614   111 29892  6843   229 14614   113  6843   122  6843
   102 21507 14614   122  9916   229  6843   229     3]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:

segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];


In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
segment_ids = torch.tensor(segment_ids)

In [ ]:
model.eval();

In [ ]:
with torch.no_grad():
        outputs = save_model(input_ids, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

In [ ]:
predict_results = logits.detach().cpu().numpy()
predict_results.shape


(1, 45, 30522)

In [ ]:
from scipy.special import softmax
result_arrays_soft = softmax(predict_results[0])
result_arrays_soft[0]


array([3.4210159e-13, 1.8820103e-13, 5.8559746e-10, ..., 1.5407079e-14,
       1.5783559e-13, 4.6627822e-14], dtype=float32)

In [ ]:
result_array = result_arrays_soft
len(result_array),len(result_array[0])

(45, 30522)

In [ ]:
# Get each token final predict tag index result
result_list = np.argmax(result_array,axis=-1)
result_list

array([43, 15, 41, 41, 41, 41, 41, 22, 13, 41, 41, 41, 41, 42, 41, 41,  0,
       41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 41, 41, 41, 41, 42, 41, 41,
       41, 41, 41, 41, 42, 41, 41, 41, 41, 41, 44])

In [ ]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
#         print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()

Token:[CLS]
Predict_Tag:[CLS]

Token:à¤
Predict_Tag:B-PERSON

Token:®
Predict_Tag:X

Token:à¥
Predict_Tag:X

Token:Ī
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:Ĥ
Predict_Tag:X

Token:Ġ20
Predict_Tag:B-COUNT

Token:Ġà¤
Predict_Tag:I-PERIOD

Token:¸
Predict_Tag:X

Token:à¤¾
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:²
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:¸
Predict_Tag:X

Token:à¥ĩ
Predict_Tag:X

Token:Ġà¤
Predict_Tag:B-LOCATION

Token:¦
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:¿
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:²
Predict_Tag:X

Token:à¥į
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:²
Predict_Tag:X

Token:à¥Ģ
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:®
Predict_Tag:X

Token:à¥ĩ
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:Ĥ
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:°
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:¹
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:¤
Predict_Tag:X

Token:à¤¾
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:¹
Predict_Tag:X

Token:à